<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


```jupyter
!pip3 install folium
!pip3 install wget
```

In [1]:
import folium
import requests
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


**wget is problematic for me locally. Pandas supports reading csv URL directly**

In [3]:
# Download and read the `spacex_launch_geo.csv`
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

CSV_URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(CSV_URL)


Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
#marker = folium.map.Marker(
marker = folium.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


So, let's see the actual names of the sites

In [7]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [8]:
# One way of getting the average lat and long in the table for the places, grouping by and performing mean operation.
# As a side effect, we get the mean values for the success class column in each launch site.
spacex_df.groupby('Launch Site').mean()

,Lat,Long,class
Launch Site,,,
CCAFS LC-40,28.562302,-80.577356,0.269231
CCAFS SLC-40,28.563197,-80.576820,0.428571
KSC LC-39A,28.573255,-80.646895,0.769231
VAFB SLC-4E,34.632834,-120.610745,0.400000


In [9]:
grouped_df = spacex_df.groupby('Launch Site').mean()
for place in spacex_df['Launch Site'].unique().tolist():
    long,lat = grouped_df.loc[place][['Long','Lat']]
    print(f'{long = } - {lat = }')

long = -80.57735648 - lat = 28.56230197
long = -120.61074549999998 - lat = 34.63283416
long = -80.64689529 - lat = 28.57325457
long = -80.57682003 - lat = 28.56319718


As there are 4 sites, I'll ad 4 colors

In [10]:
from matplotlib.colors import cnames as named_color_dictionary
site_color = {site:named_color_dictionary[color] for site,color in zip(spacex_df['Launch Site'].unique().tolist(),['black','crimson','dodgerblue','darkviolet'])}
site_color

{'CCAFS LC-40': '#000000',
 'VAFB SLC-4E': '#DC143C',
 'KSC LC-39A': '#1E90FF',
 'CCAFS SLC-40': '#9400D3'}

In [11]:
# Initializing the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
grouped_df = spacex_df.groupby('Launch Site').mean()
for place in spacex_df['Launch Site'].unique().tolist():
    long,lat = grouped_df.loc[place][['Long','Lat']]
    
    site_map.add_child(
        folium.Circle([lat,long], radius=1000, color=site_color[place], fill=True).add_child(folium.Popup(f'{place}'))
    )
    site_map.add_child(
        folium.Marker(
            [lat,long],
            # Create an icon as a text label
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 12; color:{site_color[place]};"><b>{place}</b></div>',
            )
        )
    )

In [12]:
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


**Answers**

+ Well, at least in the continental USA, all launch sites are as close to the equator as they can be (quite close to the USA - Mexico frontier).  
In principle, this ensures easier orbital insertion for some projected orbits

+ All **launch sites are close to the coast**. The reason is quite simple, if anything were to go wrong, the boosters are more likely to fall into the ocean, minimizing possible property or personal damage.
    
    + **KSC-LC 39A**, **CCAFS-SLC 40** and **CCAFS-LC 40** are by the athlantic ocean (Florida coast).
    + **VAFB-SLC 4E** is by the pacific ocean coast (California coast). 

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:
launch_sites = spacex_df.copy(deep=True)

color_outcome = ['green' if element == 1 else 'red' for element in launch_sites['class']]

In [16]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [17]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    lat,long = record[['Lat','Long']]
    # getting colors in the hex form 
    
    record['Launch Site']
    marker = folium.Marker([lat,long],popup=record['Launch Site'], icon=folium.Icon(color=record['marker_color'])) 
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
spacex_df['Launch Site'].unique().tolist()

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [22]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


I'm creating lat and long columns for the closest landmarks to be included

In [23]:
coastline_lats = [
    28.56228,
    28.56304,
    28.58452,
    34.63561
]

coastline_longs = [
    -80.56782,
    -80.56799,
    -80.61498,
    -120.62509
]

launch_sites_df['CoastLine_Lat'] = coastline_lats
launch_sites_df['CoastLine_Lon'] = coastline_longs
launch_sites_df

,Launch Site,Lat,Long,CoastLine_Lat,CoastLine_Lon
0,CCAFS LC-40,28.562302,-80.577356,28.56228,-80.56782
1,CCAFS SLC-40,28.563197,-80.576820,28.56304,-80.56799
2,KSC LC-39A,28.573255,-80.646895,28.58452,-80.61498
3,VAFB SLC-4E,34.632834,-120.610745,34.63561,-120.62509


In [24]:
launch_sites_df.dtypes

Launch Site       object
Lat              float64
Long             float64
CoastLine_Lat    float64
CoastLine_Lon    float64
dtype: object

In [25]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163

###########################################

list_dictances = []
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record['Lat']
    launch_site_lon = record['Long']
    coastline_lat = record['CoastLine_Lat']
    coastline_lon = record['CoastLine_Lon']

    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
    list_dictances.append(distance_coastline)

launch_sites_df['CoastLine_distance'] = list_dictances
launch_sites_df


,Launch Site,Lat,Long,CoastLine_Lat,CoastLine_Lon,CoastLine_distance
0,CCAFS LC-40,28.562302,-80.577356,28.56228,-80.56782,0.931650
1,CCAFS SLC-40,28.563197,-80.576820,28.56304,-80.56799,0.862802
2,KSC LC-39A,28.573255,-80.646895,28.58452,-80.61498,3.359815
3,VAFB SLC-4E,34.632834,-120.610745,34.63561,-120.62509,1.348622


*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


**I'm doing all of them at once, given that I created a support dataframe for the individual launch sites**

In [26]:
launch_sites_df[['Launch Site','CoastLine_distance']]

,Launch Site,CoastLine_distance
0,CCAFS LC-40,0.931650
1,CCAFS SLC-40,0.862802
2,KSC LC-39A,3.359815
3,VAFB SLC-4E,1.348622


In [27]:
coast_markers = []
for idx, record in launch_sites_df.iterrows():
    l_site = record['Launch Site']
    distance = record['CoastLine_distance']
    tag = f'{distance:10.2f} Km'
    coordinates = [record['CoastLine_Lat'], record['CoastLine_Lon']]
    color = site_color[l_site]

    coast_markers.append(folium.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{color};"><b>{tag}</b></div>',
            )
        )
    )


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [28]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = []
for idx, record in launch_sites_df.iterrows():
    coordinates = [(record['Lat'],record['Long']),(record['CoastLine_Lat'], record['CoastLine_Lon'])]
    lines.append(
        folium.PolyLine(coordinates, weight=1, color = 'black')
    )

In [29]:
# Updating everytihng at once

for marker in coast_markers:
    marker.add_to(site_map)

for line in lines:
    line.add_to(site_map)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


As before, I'm gonna do them all at once, from list recopilated from the map coordinates

In [31]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

#Railways
railway_lats = [
    28.57205,
    28.57205,
    28.64484,
    34.63544
]

railway_longs = [
    -80.5853,
    -80.5853,
    -80.69707,
    -120.62412
]

highways_lats = [
    28.56364,
    28.56364,
    28.57333,
    34.64094
]

highways_longs = [
    -80.57112,
    -80.57122,
    -80.65541,
    -120.43587
]

town_lats = [
    28.40083,
    28.40083,
    28.57667,
    34.63801
]

town_longs = [
    -80.60467,
    -80.60467,
    -80.80221,
    -120.45944
]

town_names = [
    'Cape Canaveral',
    'Cape Canaveral',
    'Titusville',
    'Lompoc'

]

launch_sites_df['Railway_Lat'] = railway_lats
launch_sites_df['Railway_Lon'] = railway_longs
launch_sites_df['Highway_Lat'] = highways_lats
launch_sites_df['Highway_Lon'] = highways_longs
launch_sites_df['Town_Lat'] = town_lats
launch_sites_df['Town_Lon'] = town_longs
launch_sites_df['Town_names'] = town_names
launch_sites_df

,Launch Site,Lat,Long,CoastLine_Lat,CoastLine_Lon,CoastLine_distance,Railway_Lat,Railway_Lon,Highway_Lat,Highway_Lon,Town_Lat,Town_Lon,Town_names
0,CCAFS LC-40,28.562302,-80.577356,28.56228,-80.56782,0.931650,28.57205,-80.58530,28.56364,-80.57112,28.40083,-80.60467,Cape Canaveral
1,CCAFS SLC-40,28.563197,-80.576820,28.56304,-80.56799,0.862802,28.57205,-80.58530,28.56364,-80.57122,28.40083,-80.60467,Cape Canaveral
2,KSC LC-39A,28.573255,-80.646895,28.58452,-80.61498,3.359815,28.64484,-80.69707,28.57333,-80.65541,28.57667,-80.80221,Titusville
3,VAFB SLC-4E,34.632834,-120.610745,34.63561,-120.62509,1.348622,34.63544,-120.62412,34.64094,-120.43587,34.63801,-120.45944,Lompoc


In [32]:
#Railways

list_dictances = []
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record['Lat']
    launch_site_lon = record['Long']
    obj_lat = record['Railway_Lat']
    obj_lon = record['Railway_Lon']

    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, obj_lat, obj_lon)
    list_dictances.append(distance_coastline)

launch_sites_df['Railway_distance'] = list_dictances

list_dictances = []
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record['Lat']
    launch_site_lon = record['Long']
    obj_lat = record['Highway_Lat']
    obj_lon = record['Highway_Lon']

    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, obj_lat, obj_lon)
    list_dictances.append(distance_coastline)

launch_sites_df['Highway_distance'] = list_dictances

list_dictances = []
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record['Lat']
    launch_site_lon = record['Long']
    obj_lat = record['Town_Lat']
    obj_lon = record['Town_Lon']

    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, obj_lat, obj_lon)
    list_dictances.append(distance_coastline)

launch_sites_df['Town_distance'] = list_dictances


launch_sites_df

,Launch Site,Lat,Long,CoastLine_Lat,CoastLine_Lon,CoastLine_distance,Railway_Lat,Railway_Lon,Highway_Lat,Highway_Lon,Town_Lat,Town_Lon,Town_names,Railway_distance,Highway_distance,Town_distance
0,CCAFS LC-40,28.562302,-80.577356,28.56228,-80.56782,0.931650,28.57205,-80.58530,28.56364,-80.57112,28.40083,-80.60467,Cape Canaveral,1.333344,0.627171,18.157932
1,CCAFS SLC-40,28.563197,-80.576820,28.56304,-80.56799,0.862802,28.57205,-80.58530,28.56364,-80.57122,28.40083,-80.60467,Cape Canaveral,1.286803,0.549290,18.264173
2,KSC LC-39A,28.573255,-80.646895,28.58452,-80.61498,3.359815,28.64484,-80.69707,28.57333,-80.65541,28.57667,-80.80221,Titusville,9.349108,0.831782,15.176075
3,VAFB SLC-4E,34.632834,-120.610745,34.63561,-120.62509,1.348622,34.63544,-120.62412,34.64094,-120.43587,34.63801,-120.45944,Lompoc,1.257879,16.029393,13.859177


In [33]:
#MARKERS
markers = []
for idx, record in launch_sites_df.iterrows():
    l_site = record['Launch Site']
    if l_site != 'CCAFS SLC-40': #because all landmarks are the same for CCAFS SLC-40 and CCAFS LC-40
        for key in ['Railway_distance','Highway_distance', 'Town_distance']:
            distance = record[key]
            # FOr the lat and long keys, let's do a bit of string manipulation
            root_key = key.split('_')[0]
            tag = f'{distance:10.2f} Km'
            coordinates = [record[f'{root_key}_Lat'], record[f'{root_key}_Lon']]
            color = site_color[l_site]

            markers.append(folium.Marker(
                coordinates,
                icon=DivIcon(
                    icon_size=(20,20),
                    icon_anchor=(0,0),
                    html=f'<div style="font-size: 12; color:{color};"><b>{tag}</b></div>',
                    )
                )
            )

# LINES
lines = []
for idx, record in launch_sites_df.iterrows():
    l_site = record['Launch Site']
    for key_root in ['Railway','Highway','Town']:
        coordinates = [(record['Lat'],record['Long']),(record[f'{key_root}_Lat'], record[f'{key_root}_Lon'])]
        color = site_color[l_site]
        lines.append(
            folium.PolyLine(coordinates, weight=1, color = color)
        )


In [34]:
for marker in markers:
    marker.add_to(site_map)

for line in lines:
    line.add_to(site_map)


site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


**Answers**

+ Yes, most launch sites are close to railways.  
    *Basically, the boosters can be delivered by train from many other places in the country*

+ Some launch sites are close to highways.  
    *The motive is shared with the railways, easy delivery*

+ Coasline proximity is a yes as well.  
    *Already explained above, in its dedicated section.*

+ Cities tend to keep a distance from cities.  
    *In the case of a malfunction, nobody wants a booster falling on them ...  
    The closest town to our list of launch complexes is **Lompoc**, with respect to **VAFB SLC-4E** (which is also the only complex without a clear access to a highway)*

### DISCLAIMER
To visualie correctly this notebooks, given that folium is not a trusted source for GitHub, y recomend pasting the url for this page into :

[nbviewer](https://nbviewer.org/)

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
